In [1]:
from lxml import etree
import re 

et = etree.parse("evpn_data.xml")
#print(etree.tostring(et, pretty_print=True).decode())

In [2]:
# get existed vlans with vxlan (accEncap attribute) 
# and init evpn_data object
evpn_data = []
bd_list = et.xpath('.//BD-list[.//accEncap]')

for bd in bd_list:
    evpn = dict()
    evpn["vlan"] = bd.findtext("fabEncap").split("-")[1]
    evpn["vni"] = bd.findtext("accEncap").split("-")[1]
    evpn_data.append(evpn)
print(evpn_data)


[{'vlan': '20', 'vni': '10020'}, {'vlan': '30', 'vni': '10030'}, {'vlan': '101', 'vni': '900001'}]


In [16]:
# enrich evpn_data with svi opts like mtu and vrf
for item in evpn_data:    
    vlan = f'vlan{item["vlan"]}'
    svi = et.xpath(f'.//svi-items/If-list[.//id="{vlan}"]')[0]
    item["mtu"] = svi.findtext("mtu")
    vrf = svi.xpath('.//rtvrfMbr-items/tDn/text()')[0]
    item["vrf"] = re.split('[\[\]]', vrf)[1].split('=')[1].strip('\'')
print(evpn_data)

[{'vlan': '20', 'vni': '10020', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': '10.0.20.254/24'}, {'vlan': '30', 'vni': '10030', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': '10.0.30.254/24'}, {'vlan': '101', 'vni': '900001', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': []}]


In [28]:
ipv4item = et.xpath(f'.//if-items/If-list[.//id="vlan20"]')[0]
ip_address = ipv4item.xpath('.//addr-items/Addr-list/addr/text()')

type(ip_address)


list

In [29]:
# enrich evpn_data with ipv4 data
for item in evpn_data:    
    vlan = f'vlan{item["vlan"]}'
    ipv4item = et.xpath(f'.//if-items/If-list[.//id="{vlan}"]')[0]
    item["ip_address"] = ipv4item.xpath('.//addr-items/Addr-list/addr/text()')
    
print(evpn_data)


[{'vlan': '20', 'vni': '10020', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': ['10.0.20.254/24']}, {'vlan': '30', 'vni': '10030', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': ['10.0.30.254/24']}, {'vlan': '101', 'vni': '900001', 'mtu': '1500', 'vrf': 'Tenant-1', 'ip_address': []}]


In [24]:
# filter out l2vni's
vni_list = et.xpath('.//Nw-list[.//associateVrfFlag="false"]')

for vni in vni_list:
    print (f'vni: {vni.findtext("vni")}; mcast: {vni.findtext("mcastGroup")}')


vni: 10030; mcast: 230.1.1.1
vni: 10020; mcast: 230.1.1.1
